In [7]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
import pandas as pd
import numpy as np
import plotly.express as px


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

server=app.server
'''
Gathering the data.
For this assignment we had to use an API.
'''
bronx_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id),health,steward,boroname' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=spc_common,health,steward,boroname').replace(' ', '%20')
bronx_df = pd.read_json(bronx_url)
bronx_df= bronx_df.dropna()
man_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id),health,steward,boroname' +\
        '&$where=boroname=\'Manhattan\'' +\
        '&$group=spc_common,health,steward,boroname').replace(' ', '%20')
man_df = pd.read_json(man_url)
man_df = man_df.dropna()
queens_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id),health,steward,boroname' +\
        '&$where=boroname=\'Queens\'' +\
        '&$group=spc_common,health,steward,boroname' +\
        '&$limit=2000').replace(' ', '%20')
queens_df = pd.read_json(queens_url)
queens_df = queens_df.dropna()
brooklyn_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id), health, steward,boroname' +\
        '&$where=boroname=\'Brooklyn\'' +\
        '&$group=spc_common, health, steward, boroname' +\
        '&$limit=2000').replace(' ', '%20')
brooklyn_df = pd.read_json(brooklyn_url)
brooklyn_df = brooklyn_df.dropna()
si_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id),health,steward,boroname' +\
        '&$where=boroname=\'Staten Island\'' +\
        '&$group=spc_common,health,steward,boroname').replace(' ', '%20')
si_df = pd.read_json(si_url)
si_df = si_df.dropna()
frames = [bronx_df, man_df, queens_df, brooklyn_df, si_df]
trees_df = pd.concat(frames)
trees_df

fig_two = px.histogram(trees_df, x='health', y='count_tree_id', color='steward', barmode='group')

app.layout = html.Div([
    
    html.H1("New York City Trees", style={'text-align':'center'}),
    html.H2("Tyler Baker", style={'text-align': 'center'}),
    
    html.H3("How healthy is a tree species in a given borugh?", style={'text-align': 'center'}),
    
    html.Div([
        dcc.Dropdown(id="boro",
                     options=trees_df['boroname'].unique(),
                     value="Manhattan")
    ]),
    
    html.Div([
        dcc.Dropdown(id='species',
                    options=trees_df['spc_common'].unique(),
                    value="American beech")
    ]),
 
    html.Div([
        dcc.Graph(id='graph_one')
    ]),
    
    html.Br(),
    
    html.H4("Do stewards help?", style={'text-align':'center'}),
        
    html.Div([
        dcc.Graph(id='graph_two',
                 figure=fig_two)
    ])
    
])

@app.callback(
    dash.dependencies.Output('graph_one', 'figure'),
    
    dash.dependencies.Input('boro', 'value'),
    dash.dependencies.Input('species', 'value'))
def update_fig_one(sel_boro, sel_spec):
    df_one = trees_df[trees_df['boroname']==sel_boro]
    df_one = df_one[df_one['spc_common']==sel_spec]
    tree_count_one = df_one['count_tree_id'].sum()
    health_one = df_one[df_one['health']=="Good"]
    health_one_per = ((health_one['count_tree_id'].sum())/(tree_count_one))*100
    fair_one = df_one[df_one['health']=="Fair"]
    fair_one_per = ((fair_one['count_tree_id'].sum())/(tree_count_one))*100
    poor_one = df_one[df_one['health']=="Poor"]
    poor_one_per = ((poor_one['count_tree_id'].sum())/(tree_count_one))*100
    
    bits = [["Good", health_one_per], ["Fair", fair_one_per],["Poor", poor_one_per]]
    dff = pd.DataFrame(bits, columns = ["health", "percentages"])
    
    fig= px.bar(dff, x='health', y='percentages')
    fig.update_layout(transition_duration=500)
    return fig
    





  

    
    

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
